In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = "../data_final_project/KuaiRec 2.0/data/"
small_matrix = pd.read_csv(path + "small_matrix.csv")
big_matrix = pd.read_csv(path + "big_matrix.csv")
item_categories = pd.read_csv(path + "item_categories.csv")
item_features = pd.read_csv(path + "item_daily_features.csv")
social_network = pd.read_csv(path + "social_network.csv")
user_features = pd.read_csv(path + "user_features.csv")

In [3]:
try:
    # Method 1: Using Python engine instead of C engine
    captions = pd.read_csv(path + "kuairec_caption_category.csv", engine='python')
    print("Captions loaded successfully with Python engine")
except Exception as e:
    print(f"Method 1 failed: {e}")
    try:
        # Method 2: Increase buffer size and skip bad lines
        captions = pd.read_csv(path + "kuairec_caption_category.csv", 
                              engine='python', 
                              on_bad_lines='skip',
                              encoding='utf-8')
        print("Captions loaded successfully with skipping bad lines")
    except Exception as e:
        print(f"Method 2 failed: {e}")
        print("Unable to load captions file. Proceeding without it.")
        captions = None

Captions loaded successfully with Python engine


In [4]:
sns.set(font_scale=1.2)
pd.set_option('display.max_columns', None)

In [5]:
print(f"Small matrix shape: {small_matrix.shape}")
print(f"Big matrix shape: {big_matrix.shape}")
print(f"Item categories shape: {item_categories.shape}")
print(f"Item features shape: {item_features.shape}")
print(f"Social network shape: {social_network.shape}")
print(f"User features shape: {user_features.shape}")
print(f"Captions shape: {captions.shape}")


Small matrix shape: (4676570, 8)
Big matrix shape: (12530806, 8)
Item categories shape: (10728, 2)
Item features shape: (343341, 58)
Social network shape: (472, 2)
User features shape: (7176, 31)
Captions shape: (10732, 10)


In [6]:
def display_dataset_info(df, name):
    print(f"\n=== {name} Dataset ===")
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print("\nData Types:")
    print(df.dtypes)
    print("\nSample Data:")
    display(df.head())
    print("\nBasic Statistics:")
    display(df.describe())
    
    # Check for missing values
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print("\nMissing Values:")
        display(missing[missing > 0])
    else:
        print("\nNo missing values found.")
        
    # Check for duplicates
    duplicates = df.duplicated().sum()
    print(f"\nDuplicate rows: {duplicates}")
    
    print("\n" + "="*50)

In [7]:
display_dataset_info(small_matrix, "Small Matrix")


=== Small Matrix Dataset ===
Shape: (4676570, 8)
Columns: ['user_id', 'video_id', 'play_duration', 'video_duration', 'time', 'date', 'timestamp', 'watch_ratio']

Data Types:
user_id             int64
video_id            int64
play_duration       int64
video_duration      int64
time               object
date              float64
timestamp         float64
watch_ratio       float64
dtype: object

Sample Data:


,user_id,video_id,play_duration,video_duration,time,date,timestamp,watch_ratio
0,14,148,4381,6067,2020-07-05 05:27:48.378,20200705.0,1.593898e+09,0.722103
1,14,183,11635,6100,2020-07-05 05:28:00.057,20200705.0,1.593898e+09,1.907377
2,14,3649,22422,10867,2020-07-05 05:29:09.479,20200705.0,1.593898e+09,2.063311
3,14,5262,4479,7908,2020-07-05 05:30:43.285,20200705.0,1.593898e+09,0.566388
4,14,8234,4602,11000,2020-07-05 05:35:43.459,20200705.0,1.593899e+09,0.418364



Basic Statistics:


,user_id,video_id,play_duration,video_duration,date,timestamp,watch_ratio
count,4.676570e+06,4.676570e+06,4.676570e+06,4.676570e+06,4.494578e+06,4.494578e+06,4.676570e+06
mean,3.631649e+03,4.974939e+03,8.612637e+03,1.448645e+04,2.020077e+07,1.596241e+09,9.070695e-01
std,2.043873e+03,3.064622e+03,1.223661e+04,2.046711e+04,4.895180e+01,1.254444e+06,1.362324e+00
min,1.400000e+01,1.030000e+02,0.000000e+00,3.067000e+03,2.020070e+07,1.593801e+09,0.000000e+00
25%,1.834000e+03,2.369000e+03,5.811000e+03,7.523000e+03,2.020072e+07,1.595210e+09,4.675769e-01
50%,3.687000e+03,4.692000e+03,7.549000e+03,9.600000e+03,2.020080e+07,1.596224e+09,7.691666e-01
75%,5.421000e+03,7.474000e+03,9.880000e+03,1.193400e+04,2.020081e+07,1.597121e+09,1.120590e+00
max,7.162000e+03,1.059500e+04,7.988155e+06,3.150720e+05,2.020090e+07,1.599321e+09,5.715214e+02



Missing Values:


time         181992
date         181992
timestamp    181992
dtype: int64


Duplicate rows: 0



In [8]:
display_dataset_info(big_matrix, "Big Matrix")


=== Big Matrix Dataset ===
Shape: (12530806, 8)
Columns: ['user_id', 'video_id', 'play_duration', 'video_duration', 'time', 'date', 'timestamp', 'watch_ratio']

Data Types:
user_id             int64
video_id            int64
play_duration       int64
video_duration      int64
time               object
date                int64
timestamp         float64
watch_ratio       float64
dtype: object

Sample Data:


,user_id,video_id,play_duration,video_duration,time,date,timestamp,watch_ratio
0,0,3649,13838,10867,2020-07-05 00:08:23.438,20200705,1.593879e+09,1.273397
1,0,9598,13665,10984,2020-07-05 00:13:41.297,20200705,1.593879e+09,1.244082
2,0,5262,851,7908,2020-07-05 00:16:06.687,20200705,1.593879e+09,0.107613
3,0,1963,862,9590,2020-07-05 00:20:26.792,20200705,1.593880e+09,0.089885
4,0,8234,858,11000,2020-07-05 00:43:05.128,20200705,1.593881e+09,0.078000



Basic Statistics:


,user_id,video_id,play_duration,video_duration,date,timestamp,watch_ratio
count,1.253081e+07,1.253081e+07,1.253081e+07,1.253081e+07,1.253081e+07,1.253081e+07,1.253081e+07
mean,3.574377e+03,5.057748e+03,9.027027e+03,1.462157e+04,2.020080e+07,1.596799e+09,9.445059e-01
std,2.067008e+03,3.089868e+03,1.547343e+04,1.983474e+04,5.080192e+01,1.514698e+06,1.674601e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,1.400000e+02,2.020070e+07,1.592872e+09,0.000000e+00
25%,1.788000e+03,2.387000e+03,4.218000e+03,7.434000e+03,2.020080e+07,1.596339e+09,3.148246e-01
50%,3.578000e+03,4.822000e+03,7.277000e+03,9.636000e+03,2.020081e+07,1.596669e+09,7.234710e-01
75%,5.343750e+03,7.600000e+03,1.035000e+04,1.217900e+04,2.020083e+07,1.598502e+09,1.177644e+00
max,7.175000e+03,1.072700e+04,9.996390e+05,3.150720e+05,2.020090e+07,1.599694e+09,5.734571e+02



No missing values found.

Duplicate rows: 965819



In [9]:
display_dataset_info(item_categories, "Item Categories")


=== Item Categories Dataset ===
Shape: (10728, 2)
Columns: ['video_id', 'feat']

Data Types:
video_id     int64
feat        object
dtype: object

Sample Data:


,video_id,feat
0,0,[8]
1,1,"[27, 9]"
2,2,[9]
3,3,[26]
4,4,[5]



Basic Statistics:


,video_id
count,10728.000000
mean,5363.500000
std,3097.051178
min,0.000000
25%,2681.750000
50%,5363.500000
75%,8045.250000
max,10727.000000



No missing values found.

Duplicate rows: 0



In [10]:
display_dataset_info(item_features, "Item Features")


=== Item Features Dataset ===
Shape: (343341, 58)
Columns: ['video_id', 'date', 'author_id', 'video_type', 'upload_dt', 'upload_type', 'visible_status', 'video_duration', 'video_width', 'video_height', 'music_id', 'video_tag_id', 'video_tag_name', 'show_cnt', 'show_user_num', 'play_cnt', 'play_user_num', 'play_duration', 'complete_play_cnt', 'complete_play_user_num', 'valid_play_cnt', 'valid_play_user_num', 'long_time_play_cnt', 'long_time_play_user_num', 'short_time_play_cnt', 'short_time_play_user_num', 'play_progress', 'comment_stay_duration', 'like_cnt', 'like_user_num', 'click_like_cnt', 'double_click_cnt', 'cancel_like_cnt', 'cancel_like_user_num', 'comment_cnt', 'comment_user_num', 'direct_comment_cnt', 'reply_comment_cnt', 'delete_comment_cnt', 'delete_comment_user_num', 'comment_like_cnt', 'comment_like_user_num', 'follow_cnt', 'follow_user_num', 'cancel_follow_cnt', 'cancel_follow_user_num', 'share_cnt', 'share_user_num', 'download_cnt', 'download_user_num', 'report_cnt', 'r

,video_id,date,author_id,video_type,upload_dt,upload_type,visible_status,video_duration,video_width,video_height,music_id,video_tag_id,video_tag_name,show_cnt,show_user_num,play_cnt,play_user_num,play_duration,complete_play_cnt,complete_play_user_num,valid_play_cnt,valid_play_user_num,long_time_play_cnt,long_time_play_user_num,short_time_play_cnt,short_time_play_user_num,play_progress,comment_stay_duration,like_cnt,like_user_num,click_like_cnt,double_click_cnt,cancel_like_cnt,cancel_like_user_num,comment_cnt,comment_user_num,direct_comment_cnt,reply_comment_cnt,delete_comment_cnt,delete_comment_user_num,comment_like_cnt,comment_like_user_num,follow_cnt,follow_user_num,cancel_follow_cnt,cancel_follow_user_num,share_cnt,share_user_num,download_cnt,download_user_num,report_cnt,report_user_num,reduce_similar_cnt,reduce_similar_user_num,collect_cnt,collect_user_num,cancel_collect_cnt,cancel_collect_user_num
0,0,20200705,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,3350323409,841,建筑,14665,11372,10141,7485,88729488,5657,4834,5503,4775,5503,4775,1939,1481,0.799860,6629173,573,569,315,257,87,85,11,11,8,3,0,0,112,61,284,284,0,0,2,2,8,8,0,0,3,3,NaN,NaN,NaN,NaN
1,0,20200706,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,3350323409,841,建筑,10883,8513,7321,5490,64264607,4162,3522,4039,3468,4039,3468,1340,1040,0.805253,3997498,302,301,159,142,47,47,7,7,6,1,0,0,60,32,201,200,0,0,1,1,2,2,0,0,5,5,NaN,NaN,NaN,NaN
2,0,20200707,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,3350323409,841,建筑,7842,6281,4757,3724,41338741,2734,2403,2640,2376,2640,2376,866,683,0.808821,3314323,205,205,121,84,52,50,4,3,3,1,0,0,59,26,131,131,0,0,1,1,2,2,0,0,0,0,NaN,NaN,NaN,NaN
3,0,20200708,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,3350323409,841,建筑,8916,7229,5172,3961,45281254,2950,2525,2865,2498,2865,2498,977,765,0.801680,4235579,297,293,178,119,60,59,4,4,2,2,0,0,91,46,179,179,0,0,2,2,3,3,0,0,3,3,NaN,NaN,NaN,NaN
4,0,20200709,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,3350323409,841,建筑,8502,6658,5392,3946,46952744,3058,2566,2946,2533,2945,2532,1046,752,0.805359,3862095,307,305,166,141,57,56,5,2,0,5,0,0,76,47,186,186,0,0,0,0,2,2,2,1,1,1,NaN,NaN,NaN,NaN



Basic Statistics:


,video_id,date,author_id,video_duration,video_width,video_height,music_id,video_tag_id,show_cnt,show_user_num,play_cnt,play_user_num,play_duration,complete_play_cnt,complete_play_user_num,valid_play_cnt,valid_play_user_num,long_time_play_cnt,long_time_play_user_num,short_time_play_cnt,short_time_play_user_num,play_progress,comment_stay_duration,like_cnt,like_user_num,click_like_cnt,double_click_cnt,cancel_like_cnt,cancel_like_user_num,comment_cnt,comment_user_num,direct_comment_cnt,reply_comment_cnt,delete_comment_cnt,delete_comment_user_num,comment_like_cnt,comment_like_user_num,follow_cnt,follow_user_num,cancel_follow_cnt,cancel_follow_user_num,share_cnt,share_user_num,download_cnt,download_user_num,report_cnt,report_user_num,reduce_similar_cnt,reduce_similar_user_num,collect_cnt,collect_user_num,cancel_collect_cnt,cancel_collect_user_num
count,343341.000000,3.433410e+05,343341.000000,332743.000000,343341.000000,343341.000000,3.433410e+05,343341.000000,3.433410e+05,3.433410e+05,3.433410e+05,3.433410e+05,3.433410e+05,3.433410e+05,3.433410e+05,3.433410e+05,3.433410e+05,3.433410e+05,3.433410e+05,3.433410e+05,3.433410e+05,343341.000000,3.433410e+05,3.433410e+05,3.433410e+05,3.433410e+05,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,343341.000000,273658.000000,273658.000000,273658.000000,273658.000000
mean,5077.370157,2.020080e+07,4220.959932,12508.367091,713.117833,1218.797452,3.041715e+09,1363.171585,9.424580e+04,8.859975e+04,9.418781e+04,8.483503e+04,1.804975e+09,5.075386e+04,4.824550e+04,6.195622e+04,5.919886e+04,5.292314e+04,5.079426e+04,1.875147e+04,1.688551e+04,0.379127,9.853639e+07,2.781109e+03,2.757480e+03,1.371501e+03,1406.157063,178.559278,168.600135,130.480764,114.464416,96.688115,33.792649,4.858421,4.083762,749.788988,351.127486,302.643914,302.488034,0.023158,0.022610,80.380860,74.785950,44.557484,39.704049,0.042780,0.037549,50.422289,48.942605,20.014686,19.837754,1.320338,1.297514
std,3113.616949,5.654698e+01,2390.317222,13904.816578,122.673938,165.715482,1.543376e+09,1090.364659,5.761700e+05,5.512458e+05,5.933298e+05,5.413591e+05,1.605857e+10,3.563492e+05,3.442709e+05,4.177207e+05,4.049903e+05,3.672900e+05,3.579913e+05,1.152964e+05,1.054209e+05,0.275441,1.199933e+09,1.869603e+04,1.850841e+04,9.628631e+03,9454.005871,1557.676774,1449.526371,1565.610155,1361.543208,1225.041919,506.862290,134.200906,87.218102,7628.812713,3379.499898,2822.293469,2820.667128,0.277330,0.267823,1284.173127,1186.872229,535.575910,471.296386,0.822401,0.695511,281.581431,273.243597,356.678666,353.742019,7.581929,7.364914
min,0.000000,2.020070e+07,0.000000,40.000000,270.000000,320.000000,0.000000e+00,-124.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2318.000000,2.020073e+07,2126.000000,6901.000000,720.000000,1280.000000,2.354487e+09,144.000000,5.500000e+01,4.700000e+01,1.400000e+01,1.200000e+01,1.017910e+05,4.000000e+00,4.000000e+00,5.000000e+00,5.000000e+00,4.000000e+00,4.000000e+00,6.000000e+00,5.000000e+00,0.117749,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5235.000000,2.020081e+07,4402.000000,9466.00


Missing Values:


video_duration             10598
video_tag_name             32434
collect_cnt                69683
collect_user_num           69683
cancel_collect_cnt         69683
cancel_collect_user_num    69683
dtype: int64


Duplicate rows: 0



In [11]:
display_dataset_info(social_network, "Social Network")


=== Social Network Dataset ===
Shape: (472, 2)
Columns: ['user_id', 'friend_list']

Data Types:
user_id         int64
friend_list    object
dtype: object

Sample Data:


,user_id,friend_list
0,3371,[2975]
1,24,[2665]
2,4402,[38]
3,4295,[4694]
4,7087,[7117]



Basic Statistics:


,user_id
count,472.000000
mean,3403.218220
std,2079.018308
min,18.000000
25%,1648.000000
50%,3268.000000
75%,5233.500000
max,7174.000000



No missing values found.

Duplicate rows: 0



In [12]:
display_dataset_info(user_features, "User Features")


=== User Features Dataset ===
Shape: (7176, 31)
Columns: ['user_id', 'user_active_degree', 'is_lowactive_period', 'is_live_streamer', 'is_video_author', 'follow_user_num', 'follow_user_num_range', 'fans_user_num', 'fans_user_num_range', 'friend_user_num', 'friend_user_num_range', 'register_days', 'register_days_range', 'onehot_feat0', 'onehot_feat1', 'onehot_feat2', 'onehot_feat3', 'onehot_feat4', 'onehot_feat5', 'onehot_feat6', 'onehot_feat7', 'onehot_feat8', 'onehot_feat9', 'onehot_feat10', 'onehot_feat11', 'onehot_feat12', 'onehot_feat13', 'onehot_feat14', 'onehot_feat15', 'onehot_feat16', 'onehot_feat17']

Data Types:
user_id                    int64
user_active_degree        object
is_lowactive_period        int64
is_live_streamer           int64
is_video_author            int64
follow_user_num            int64
follow_user_num_range     object
fans_user_num              int64
fans_user_num_range       object
friend_user_num            int64
friend_user_num_range     object
regist

,user_id,user_active_degree,is_lowactive_period,is_live_streamer,is_video_author,follow_user_num,follow_user_num_range,fans_user_num,fans_user_num_range,friend_user_num,friend_user_num_range,register_days,register_days_range,onehot_feat0,onehot_feat1,onehot_feat2,onehot_feat3,onehot_feat4,onehot_feat5,onehot_feat6,onehot_feat7,onehot_feat8,onehot_feat9,onehot_feat10,onehot_feat11,onehot_feat12,onehot_feat13,onehot_feat14,onehot_feat15,onehot_feat16,onehot_feat17
0,0,high_active,0,0,0,5,"(0,10]",0,0,0,0,107,61-90,0,1,17,638,2.0,0,1,6,184,6,3,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,full_active,0,0,0,386,"(250,500]",4,"[1,10)",2,"[1,5)",327,181-365,0,3,25,1021,0.0,0,1,6,186,6,2,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,full_active,0,0,0,27,"(10,50]",0,0,0,0,116,91-180,0,6,8,402,0.0,0,0,2,51,2,3,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,full_active,0,0,0,16,"(10,50]",0,0,0,0,105,61-90,0,1,8,281,0.0,0,0,34,251,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,full_active,0,0,0,122,"(100,150]",4,"[1,10)",0,0,225,181-365,0,1,8,316,1.0,0,1,46,99,4,2,0,0.0,0.0,0.0,0.0,0.0,0.0



Basic Statistics:


,user_id,is_lowactive_period,is_live_streamer,is_video_author,follow_user_num,fans_user_num,friend_user_num,register_days,onehot_feat0,onehot_feat1,onehot_feat2,onehot_feat3,onehot_feat4,onehot_feat5,onehot_feat6,onehot_feat7,onehot_feat8,onehot_feat9,onehot_feat10,onehot_feat11,onehot_feat12,onehot_feat13,onehot_feat14,onehot_feat15,onehot_feat16,onehot_feat17
count,7176.000000,7176.000000,7176.000000,7176.000000,7176.000000,7176.000000,7176.000000,7176.000000,7176.000000,7176.000000,7176.000000,7176.000000,6975.000000,7176.000000,7176.000000,7176.000000,7176.000000,7176.000000,7176.000000,7176.000000,7099.000000,7101.000000,7101.000000,7102.000000,7102.000000,7102.000000
mean,3587.500000,0.000418,0.006828,0.169593,197.327899,12.553094,4.494844,296.790691,0.392280,2.670569,15.852007,621.423634,1.051613,0.011845,0.567447,18.761148,168.661511,3.831940,2.264353,0.137124,0.298774,0.104633,0.094775,0.018586,0.017882,0.014503
std,2071.677098,0.020444,0.082357,0.375301,426.543245,181.017537,44.897861,286.381320,0.488293,1.782502,8.219267,305.770169,1.000102,0.270757,0.499945,13.118674,96.254783,1.747046,1.063131,0.500184,0.457753,0.306102,0.292925,0.135068,0.132533,0.119560
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1793.750000,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,132.000000,0.000000,1.000000,8.000000,363.000000,0.000000,0.000000,0.000000,6.000000,88.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3587.500000,0.000000,0.000000,0.000000,33.000000,2.000000,0.000000,225.000000,0.000000,2.000000,17.000000,632.000000,1.000000,0.000000,1.000000,16.000000,167.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5381.250000,0.000000,0.000000,0.000000,130.000000,6.000000,1.000000,324.000000,1.000000,4.000000,24.000000,918.000000,1.000000,0.000000,1.000000,33.000000,255.000000,5.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7175.000000,1.000000,1.000000,1.000000,2100.000000,11401.000000,1425.000000,2245.000000,1.000000,7.000000,29.000000,1075.000000,11.000000,9.000000,2.000000,46.000000,339.000000,6.000000,4.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



Missing Values:


onehot_feat4     201
onehot_feat12     77
onehot_feat13     75
onehot_feat14     75
onehot_feat15     74
onehot_feat16     74
onehot_feat17     74
dtype: int64


Duplicate rows: 0



In [13]:
display_dataset_info(captions, "Captions")


=== Captions Dataset ===
Shape: (10732, 10)
Columns: ['video_id', 'manual_cover_text', 'caption', 'topic_tag', 'first_level_category_id', 'first_level_category_name', 'second_level_category_id', 'second_level_category_name', 'third_level_category_id', 'third_level_category_name']

Data Types:
video_id                       object
manual_cover_text              object
caption                        object
topic_tag                      object
first_level_category_id       float64
first_level_category_name      object
second_level_category_id      float64
second_level_category_name     object
third_level_category_id       float64
third_level_category_name      object
dtype: object

Sample Data:


,video_id,manual_cover_text,caption,topic_tag,first_level_category_id,first_level_category_name,second_level_category_id,second_level_category_name,third_level_category_id,third_level_category_name
0,0,UNKNOWN,精神小伙路难走 程哥你狗粮慢点撒,[],8.0,颜值,673.0,颜值随拍,-124.0,UNKNOWN
1,1,UNKNOWN,NaN,[],27.0,高新数码,-124.0,UNKNOWN,-124.0,UNKNOWN
2,2,UNKNOWN,晚饭后，运动一下！,[],9.0,喜剧,727.0,搞笑互动,-124.0,UNKNOWN
3,3,UNKNOWN,我平淡无奇，惊艳不了时光，温柔不了岁月，我只想漫无目的的走走，努力发笔小财，给自己买花 自己长大.,[],26.0,摄影,686.0,主题摄影,2434.0,景物摄影
4,4,五爱街最美美女 一天1q,#搞笑 #感谢快手我要上热门 #五爱市场 这真是完美搭配啊！,"[五爱市场,感谢快手我要上热门,搞笑]",5.0,时尚,737.0,营销售卖,2596.0,女装



Basic Statistics:


,first_level_category_id,second_level_category_id,third_level_category_id
count,10728.000000,10728.000000,10724.000000
mean,16.630593,266.661260,661.434446
std,15.853232,341.524977,1104.049489
min,-124.000000,-124.000000,-124.000000
25%,7.000000,-124.000000,-124.000000
50%,15.000000,223.000000,-124.000000
75%,28.000000,682.000000,1524.000000
max,667.000000,2377.000000,2676.000000



Missing Values:


video_id                         1
caption                       1355
topic_tag                        4
first_level_category_id          4
first_level_category_name        4
second_level_category_id         4
second_level_category_name       4
third_level_category_id          8
third_level_category_name        8
dtype: int64


Duplicate rows: 0



In [14]:
sm_users = small_matrix['user_id'].nunique()
sm_video = small_matrix['video_id'].nunique()
print(f"Small Matrix:")
print(f"  - Unique Users: {sm_users}")
print(f"  - Unique Items: {sm_video}")

Small Matrix:
  - Unique Users: 1411
  - Unique Items: 3327


In [15]:
bg_users = big_matrix['user_id'].nunique()
bg_video = big_matrix['video_id'].nunique()
print(f"Big Matrix:")
print(f"  - Unique Users: {bg_users}")
print(f"  - Unique Items: {bg_video}")

Big Matrix:
  - Unique Users: 7176
  - Unique Items: 10728


In [16]:
print(f"User Features:")
print(f"  - Total Users: {len(user_features)}")

User Features:
  - Total Users: 7176


In [17]:
print(f"Item Features:")
print(f"  - Total Items: {item_features['video_id'].nunique()}")

Item Features:
  - Total Items: 10728


In [18]:
timestamp_cols = []
if small_matrix is not None:
    timestamp_cols.extend([col for col in small_matrix.columns if 'timestamp' in col.lower()])
if big_matrix is not None:
    timestamp_cols.extend([col for col in big_matrix.columns if 'timestamp' in col.lower()])

In [19]:
if timestamp_cols:
    print(f"Timestamp columns found: {timestamp_cols}")
    
    # Analyze timestamp range for each column
    for col in timestamp_cols:
        if 'small_matrix' in locals() and col in small_matrix.columns:
            df = small_matrix
            matrix_name = "Small Matrix"
        elif 'big_matrix' in locals() and col in big_matrix.columns:
            df = big_matrix
            matrix_name = "Big Matrix"
        else:
            continue
            
        try:
            # Convert timestamp to datetime if it's numeric
            if pd.api.types.is_numeric_dtype(df[col]):
                min_time = df[col].min()
                max_time = df[col].max()
                
                # Try to convert unix timestamp
                try:
                    min_time_dt = datetime.fromtimestamp(min_time)
                    max_time_dt = datetime.fromtimestamp(max_time)
                    print(f"{matrix_name} - {col}:")
                    print(f"  - Min Time: {min_time} ({min_time_dt})")
                    print(f"  - Max Time: {max_time} ({max_time_dt})")
                    print(f"  - Range: {max_time - min_time} seconds ({(max_time_dt - min_time_dt).days} days)")
                except:
                    print(f"{matrix_name} - {col}:")
                    print(f"  - Min: {min_time}")
                    print(f"  - Max: {max_time}")
                    print(f"  - Range: {max_time - min_time}")
            else:
                print(f"{matrix_name} - {col} is not numeric. Sample values:")
                print(df[col].sample(5).tolist())
        except Exception as e:
            print(f"Error analyzing {col}: {e}")
else:
    print("No timestamp columns identified.")
    
    # Even if no timestamp columns, check item_daily_features for date information
    if item_features is not None and 'date' in item_features.columns:
        print("\nDate information found in item_daily_features:")
        min_date = item_features['date'].min()
        max_date = item_features['date'].max()
        print(f"  - Date range: {min_date} to {max_date}")

Timestamp columns found: ['timestamp', 'timestamp']


Small Matrix - timestamp:
  - Min: 1593800606.06
  - Max: 1599321443.683
  - Range: 5520837.623000145
Small Matrix - timestamp:
  - Min: 1593800606.06
  - Max: 1599321443.683
  - Range: 5520837.623000145
